In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [2]:
import pandas as pd 
import numpy as np
import nltk
import re 
import os 
import random 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv', encoding='latin1')

In [4]:
df.head(15)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
null_col = ['ScreenName', 'Location', 'TweetAt', 'Sentiment']
df2 = df.drop(null_col, axis=1)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   OriginalTweet  41157 non-null  object
dtypes: int64(1), object(1)
memory usage: 643.2+ KB


In [8]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.tokenize import RegexpTokenizer

stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')
tokenizer = RegexpTokenizer(r'\w+')
# As this dataset is fetched from twitter so it has lots of people tag in tweets
# we will remove them 
tags = r"@\w*"


def preprocess_text(sentence, stem = False):
    
    sentence = [re.sub(tags, "", sentence)]
    text = []
    for word in sentence:
        
        if word not in stopwords:
            
            if stem:
                text.append(stemmer.stem(word).lower())
            else:
                text.append(word.lower())
    return tokenizer.tokenize(" ".join(text))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
print(f"Orignal Text : {df2.OriginalTweet[4]}")
print()
print(f"Preprocessed Text : {preprocess_text(df2.OriginalTweet[4])}")





#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n

Preprocessed Text : ['me', 'ready', 'to', 'go', 'at', 'supermarket', 'during', 'the', 'covid19', 'outbreak', 'not', 'because', 'i', 'm', 'paranoid', 'but', 'because', 'my', 'food', 'stock', 'is', 'litteraly', 'empty', 'the', 'coronavirus', 'is', 'a', 'serious', 'thing', 'but', 'please', 'don', 't', 'panic', 'it', 'causes', 'shortage', 'coronavirusfrance', 'restezchezvous', 'stayathome', 'confinement', 'https', 't', 'co', 'usmualq72n']


In [10]:
df2.OriginalTweet = df2.OriginalTweet.map(preprocess_text)
df2.head()

,UserName,OriginalTweet
0,3799,"[https, t, co, ifz9fan2pa, and, https, t, co, ..."
1,3800,"[advice, talk, to, your, neighbours, family, t..."
2,3801,"[coronavirus, australia, woolworths, to, give,..."
3,3802,"[my, food, stock, is, not, the, only, one, whi..."
4,3803,"[me, ready, to, go, at, supermarket, during, t..."


In [11]:
#this is an example vocabulary just to make concept clear
sample_vocab = ['its', 'one', 'thing', 'I', 'dont', 'know', 'why', 'it', 'dosent']

In [12]:
# vocabulary of words present in dataset
data_vocab = []
for text in df2.OriginalTweet:
    for word in text:
        if word not in data_vocab:
            data_vocab.append(word)

In [13]:
#function to return one-hot representation of passed text
def get_onehot_representation(text, vocab = data_vocab):
    onehot_encoded = []
    for word in text:
        temp = [0]*len(vocab)
        temp[vocab.index(word)-1] = 1
        onehot_encoded.append(temp)
    return onehot_encoded

print("One Hot Representation for sentence \"its one thing I dont know why it doesnt\" :")
get_onehot_representation(['its', 'one', 'thing', 'I', 'dont', 'know', 'why', 'it', 'dosent'], sample_vocab)

One Hot Representation for sentence "its one thing I dont know why it doesnt" :


[[0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [14]:
print(f'Length of Vocabulary : {len(data_vocab)}')
print(f'Sample of Vocabulary : {data_vocab[302 : 312]}')

Length of Vocabulary : 70948
Sample of Vocabulary : ['16mar20', 'russia', 'surveillance', 'watchdog', 'reported', 'high', 'arctic', 'where', 'man', 'traveled']


In [15]:
sample_one_hot_rep = get_onehot_representation(df2.OriginalTweet[7], data_vocab)
print(f"Shapes of a single sentence : {np.array(sample_one_hot_rep).shape}")

Shapes of a single sentence : (17, 70948)


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

sample_bow = CountVectorizer()

# sample_corpus = [['the', 'cat', 'sat'], 
#                  ['the', 'cat', 'sat', 'in', 'the', 'hat'],
#                  ['the', 'cat', 'with', 'the', 'hat']]

sample_corpus = ["its one thing I dont know why", "it doesnt even matter how hard you try", "keep that in mind I designed this rhyme", "to explain in due time"]

sample_bow.fit(sample_corpus)

def get_bow_representation(text):
        return sample_bow.transform(text)
    
print(f"Vocabulary mapping for given sample corpus : \n {sample_bow.vocabulary_}")
print("\nBag of word Representation of sentence 'the cat cat sat in the hat'")
print(get_bow_representation(["the cat cat sat in the hat"]).toarray())

Vocabulary mapping for given sample corpus : 
 {'its': 10, 'one': 15, 'thing': 18, 'dont': 2, 'know': 12, 'why': 23, 'it': 9, 'doesnt': 1, 'even': 4, 'matter': 13, 'how': 7, 'hard': 6, 'you': 24, 'try': 22, 'keep': 11, 'that': 17, 'in': 8, 'mind': 14, 'designed': 0, 'this': 19, 'rhyme': 16, 'to': 21, 'explain': 5, 'due': 3, 'time': 20}

Bag of word Representation of sentence 'the cat cat sat in the hat'
[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [17]:
sample_bow = CountVectorizer(binary = True)

sample_corpus = ["the cat sat", "the cat sat in the hat", "the cat with the hat"]

sample_bow.fit(sample_corpus)

def get_bow_representation(text):
        return sample_bow.transform(text)
    
print(f"Vacabulary mapping for given sample corpus : \n {sample_bow.vocabulary_}")
print("\nBag of word Representation of sentence 'the the the the cat cat sat in the hat'")
print(get_bow_representation(["the the the the cat cat sat in the hat"]).toarray())

Vacabulary mapping for given sample corpus : 
 {'the': 4, 'cat': 0, 'sat': 3, 'in': 2, 'hat': 1, 'with': 5}

Bag of word Representation of sentence 'the the the the cat cat sat in the hat'
[[1 1 1 1 1 0]]
